In [17]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np

In [ ]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

# parse commandline arguments
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=10000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()


def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')


# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)

    

In [18]:

tweet = []
all_tweets = []
with open('trumptweets.json', 'r') as f:
	trump_dict = json.load(f)
	trump_tweet_list = []
	for tweet in trump_dict:
		data = tweet["text"]
		all_tweets.append(data)


In [19]:
print (all_tweets)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [22]:
# labels = all_tweets.target
true_k = np.unique(all_tweets).shape[0]

AttributeError: 'list' object has no attribute 'target'

In [23]:


print("Extracting features from the training dataset "
      "using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=opts.n_features,
                                   stop_words='english', alternate_sign=False,
                                   norm=None)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    else:
        vectorizer = HashingVectorizer(n_features=opts.n_features,
                                       stop_words='english',
                                       alternate_sign=False, norm='l2')
else:
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
                                 min_df=2, stop_words='english',
                                 use_idf=opts.use_idf)
X = vectorizer.fit_transform(all_tweets)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()


# #############################################################################
# Do the actual clustering

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000, verbose=opts.verbose)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=opts.verbose)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
# print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
# print("Adjusted Rand-Index: %.3f"
#       % metrics.adjusted_rand_score(labels, km.labels_))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, km.labels_, sample_size=1000))

# print()


if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()

Extracting features from the training dataset using a sparse vectorizer
done in 0.484789s
n_samples: 25186, n_features: 10000

Clustering sparse data with MiniBatchKMeans(batch_size=1000, init_size=1000, n_clusters=24903, n_init=1,
                verbose=False)


/Users/imacmattimacmatt/Applications/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:1676: RuntimeWarning: init_size=1000 should be larger than k=24903. Setting it to 3*k
  init_size=init_size)


done in 576.254s

Top terms per cluster:
Cluster 0: invalid removing potentially lawsuit judicial begin judicialwatch forced california million
Cluster 1: https トランプ大統領 fabled face fabulous fabrication fabricating fabricated fabricate fa
Cluster 2: make america great トランプ大統領 fabricate facebook face fabulous fabrication fabricating
Cluster 3: let practice cool professional wouldn involved saw hand amp mayor
Cluster 4: republicans wide mail voter comes potential fraud voting reason fight
Cluster 5: lopezobrador_ declaration signs joint president live whitehouse realdonaldtrump rt https
Cluster 6: arizona historic joe help win want thank extreme eyes fabricating
Cluster 7: jobs arrived discuss wisconsin maga just https fabricate fabulous fabrication
Cluster 8: rock drop kasich ads negative polls hit john soon crooked
Cluster 9: rules lose danscavino change want democrats just rt https fabled
Cluster 10: healthy favorite popular anybody hero instead troops syria announced isis
Cluster 11: 

Cluster 158: visit michigan leaving heading iowa rally big great トランプ大統領 fabricated
Cluster 159: america realdonaldtrump rt fabled facebook face fabulous fabrication fabricating fabricated
Cluster 160: guts vision fail jay powell sense reserve terrible federal factories
Cluster 161: time trillion friday hits spent stock market high dollars doesn
Cluster 162: harassment presidential トランプ大統領 fa face fabulous fabrication fabricating fabricated fabricate
Cluster 163: wallace conversation congenial version somebody explain father chris phone friend
Cluster 164: dailycaller transcript corruption shows donaldjtrumpjr trying stop trump rt https
Cluster 165: michael guest mississippi fighter congressman incredible state realdonaldtrump great rt
Cluster 166: repeal replace obamacare fabled face fabulous fabrication fabricating fabricated fabricate
Cluster 167: lamestream worse getting media https fa fabulous fabrication fabricating fabricated
Cluster 168: cernovich president years trump rt extre

Cluster 289: state improvements bus safely oklahoma usdot infrastructure helping 13 proud
Cluster 290: disgraced losers lover agent lisa insurance lyin mccabe tried peter
Cluster 291: eastern interviewed seanhannity 00 enjoy tonight foxnews realdonaldtrump rt fundraiser
Cluster 292: clewandowski_ winning realdonaldtrump rt features eyes fabulous fabrication fabricating fabricated
Cluster 293: ridiculous don https fabled facebook face fabulous fabrication fabricating fabricated
Cluster 294: stand stan chatbycc people stronger strong thank president realdonaldtrump rt
Cluster 295: medications unused 2pm takebackday 10am locations opioid location october prescription
Cluster 296: jessebwatters watching thanks rt https トランプ大統領 fa fabrication fabricating fabricated
Cluster 297: liar discredit bongino figure brennan single dan community intelligence john
Cluster 298: deals worst continue countries trade help world history united states
Cluster 299: unsanitary fingers lick lsklznezl9 dirty mi

...

Cluster 24260: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24261: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24262: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24263: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24264: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24265: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24266: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24267: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24268: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24269: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24270: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24271: bounce cybergenica vows msnbc coronavir

Cluster 24411: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24412: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24413: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24414: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24415: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24416: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24417: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24418: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24419: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24420: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24421: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24422: bounce cybergenica vows msnbc coronavir

Cluster 24552: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24553: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24554: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24555: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24556: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24557: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24558: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24559: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24560: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24561: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24562: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24563: bounce cybergenica vows msnbc coronavir

Cluster 24706: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24707: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24708: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24709: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24710: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24711: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24712: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24713: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24714: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24715: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24716: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24717: bounce cybergenica vows msnbc coronavir

Cluster 24859: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24860: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24861: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24862: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24863: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24864: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24865: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24866: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24867: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24868: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24869: bounce cybergenica vows msnbc coronavirus cnn economy foxnews trump rt
Cluster 24870: bounce cybergenica vows msnbc coronavir

In [ ]:

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

print()


if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()
        

In [5]:
# vectorizer = CountVectorizer(max_df = 0.1, min_df=3)
# # apply transformation
# tf = vectorizer.fit_transform(data["tweet_data"])
# # tf_feature_names tells us what word each column in the matric represents
# tf_feature_names = vectorizer.get_feature_names()

In [8]:
true_k = np.unique(data).shape[0]

In [9]:
# model = KMeans (n_clusters = 10)
# model.fit(tf)
# clusters=model.labels_.tolist()

In [11]:
# if opts.n_components:
#     original_space_centroids = svd.inverse_transform(km.cluster_centers_)
#     order_centroids = original_space_centroids.argsort()[:, ::-1]
# else:
#     order_centroids = km.cluster_centers_.argsort()[:, ::-1]
# terms = vectorizer.get_feature_names()
# for i in range(true_k):
#     print("Cluster %d:" % i, end='')
#     for ind in order_centroids[i, :10]:
#         print(' %s' % terms[ind], end='')
#     print()

In [12]:

import logging
from optparse import OptionParser
import sys
from time import time

In [13]:
print("Extracting features from the training dataset "
      "using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=opts.n_features,
                                   stop_words='english', alternate_sign=False,
                                   norm=None)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    else:
        vectorizer = HashingVectorizer(n_features=opts.n_features,
                                       stop_words='english',
                                       alternate_sign=False, norm='l2')
else:
    vectorizer = TfidfVectorizer(max_features=opts.n_features,
                                 stop_words='english',
                                 use_idf=opts.use_idf)
X = vectorizer.fit_transform(data)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()


Extracting features from the training dataset using a sparse vectorizer


NameError: name 'opts' is not defined

In [14]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

# parse commandline arguments
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=10000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()

Automatically created module for IPython interactive environment
Usage: ipykernel_launcher.py [options]

Options:
  -h, --help            show this help message and exit
  --lsa=N_COMPONENTS    Preprocess documents with latent semantic analysis.
  --no-minibatch        Use ordinary k-means algorithm (in batch mode).
  --no-idf              Disable Inverse Document Frequency feature weighting.
  --use-hashing         Use a hashing feature vectorizer
  --n-features=N_FEATURES
                        Maximum number of features (dimensions) to extract
                        from text.
  --verbose             Print progress reports inside k-means algorithm.


In [15]:
def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)

In [16]:
if opts.n_components:
    original_space_centroids = svd.inverse_transform(model.cluster_centers_)
    order_centroids = original_space_centroids.argsort()[:, ::-1]
else:
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

AttributeError: 'KMeans' object has no attribute 'cluster_centers_'